In [ ]:
import pickle  # Importing the pickle module for saving and loading objects (serialization)
from functools import wraps  # Importing wraps to create decorators

# File to store the serialized library state
LIBRARY_FILE = 'library_state.pkl'


In [ ]:
# Decorator to ensure only Admin users can access specific methods
def admin_required(func):
    """
    This decorator is used to restrict access to certain methods, ensuring that only users with 
    the 'Admin' role can execute the method. If a non-Admin user tries to execute the function,
    they will receive an "Access Denied" message.
    
    Decorators are a way to extend the behavior of functions without modifying their code directly.
    They make the code modular by adding functionality to existing functions dynamically.
    """
    @wraps(func)  # Preserves the original function's metadata
    def wrapper(self, *args, **kwargs):
        # Check if the current user is an Admin
        if self.current_user and self.current_user['role'] == 'Admin':
            # If the user is Admin, proceed with the function call
            return func(self, *args, **kwargs)
        else:
            # Deny access if the user is not an Admin
            print("Access Denied! Only Admins can perform this action.")
            return None
    return wrapper


In [ ]:
# Book class to represent a single book in the library
class Book:
    def __init__(self, title, author, genre):
        """
        The __init__ method is the constructor for the Book class, which initializes
        the object when an instance is created. It takes parameters like title, author, and genre
        and sets them as attributes of the book instance.
        
        - 'self' refers to the instance of the class and allows access to instance variables (attributes).
        - The 'title', 'author', and 'genre' are attributes that define the properties of a book.
        """
        self.title = title
        self.author = author
        self.genre = genre
        self.is_borrowed = False  # All books are initially not borrowed (set to False)
    
    def borrow(self):
        """
        This function allows a user to borrow a book. It checks whether the book is already borrowed
        (is_borrowed flag). If not, it updates the status to borrowed and returns a success message.
        
        Functions like these make the code modular by encapsulating behavior related to specific actions.
        """
        if not self.is_borrowed:
            self.is_borrowed = True  # Mark the book as borrowed
            return f'You have successfully borrowed "{self.title}".'
        else:
            return f'Sorry, "{self.title}" is already borrowed.'
    
    def return_book(self):
        """
        This function allows a user to return a book. If the book is borrowed, it marks it as available again.
        This function demonstrates encapsulation, where the state of the book is managed within the class.
        """
        if self.is_borrowed:
            self.is_borrowed = False  # Mark the book as returned
            return f'You have successfully returned "{self.title}".'
        else:
            return f'"{self.title}" was not borrowed.'

    def __str__(self):
        """
        The __str__ method returns a human-readable string representation of the book instance.
        It is a special method in Python that is called when an object is printed.
        
        This helps in making the object output more meaningful, showing relevant details like
        the title, author, genre, and whether the book is borrowed or not.
        """
        status = 'Available' if not self.is_borrowed else 'Borrowed'
        return f'"{self.title}" by {self.author} | Genre: {self.genre} | Status: {status}'




In [ ]:
# Library class to manage books and user roles
class Library:
    def __init__(self):
        """
        The Library class is a blueprint for the library system that stores a collection of books
        and users. The constructor initializes two important attributes:
        - 'books': a list that will hold all book objects.
        - 'users': a list to store all user information.
        - 'current_user': keeps track of the user who is currently logged in.
        
        This class demonstrates core OOP principles:
        - Encapsulation: Managing library state and user access inside the class.
        - Objects: The Library class itself is an object, and it interacts with other objects (Book).
        """
        self.books = []  # List to hold all the books in the library
        self.users = []  # List to store user information
        self.current_user = None  # Tracks the current logged-in user
    
    def add_user(self, username, role):
        """
        This function adds a new user to the library system with a specific role (Admin or Member).
        The user data is stored as a dictionary in the 'users' list.
        
        Functions like this ensure that different parts of the system can interact with users.
        """
        self.users.append({"username": username, "role": role})
        print(f"User '{username}' added with role '{role}'.")

    def login(self, username):
        """
        This function handles user login. It searches for the user in the list and sets the
        'current_user' attribute to that user. This is important for tracking who is performing
        actions in the system.
        """
        for user in self.users:
            if user['username'] == username:
                self.current_user = user
                print(f"User '{username}' logged in as {user['role']}.")
                return
        print(f"User '{username}' not found!")

    @admin_required  # Decorator ensures only Admins can add books
    def add_book(self, title, author, genre):
        """
        This function allows the Admin to add a new book to the library. It creates a new Book object
        using the title, author, and genre provided, and adds it to the 'books' list.
        
        The @admin_required decorator ensures that only users with the Admin role can execute this method.
        This shows how decorators help to dynamically control access and add extra functionality to methods.
        """
        book = Book(title, author, genre)
        self.books.append(book)
        print(f'Book "{title}" added to the library.')

    def display_books(self):
        """
        This function prints a list of all the books in the library. It iterates through the 'books' list
        and displays each book using its __str__ method.
        
        This demonstrates how objects (Book) interact with each other in a system (Library).
        """
        if self.books:
            print("\nBooks available in the library:")
            for book in self.books:
                print(book)
        else:
            print("No books in the library yet!")

    def find_book(self, title):
        """
        This helper function searches for a book by title. It is an example of code modularity, where
        small reusable functions are created to perform specific tasks (searching in this case).
        
        It returns the Book object if found, or None if not found.
        """
        for book in self.books:
            if book.title.lower() == title.lower():
                return book
        return None
    
    def borrow_book(self, title):
        """
        This function allows the current user to borrow a book. It first checks if the user is an Admin
        (Admins cannot borrow books) and then searches for the book by title.
        
        Modular functions like this help keep the code organized, where each function performs a single
        responsibility (in this case, borrowing a book).
        """
        if self.current_user and self.current_user['role'] == 'Admin':
            print("Admins cannot borrow books.")
            return
        book = self.find_book(title)
        if book:
            print(book.borrow())
        else:
            print(f'Book "{title}" not found in the library.')

    def return_book(self, title):
        """
        This function allows a user to return a borrowed book. It finds the book by title and calls
        the book's return_book method to mark it as available again.
        
        Each function in the class works together, maintaining modularity and reusability.
        """
        book = self.find_book(title)
        if book:
            print(book.return_book())
        else:
            print(f'Book "{title}" not found in the library.')




In [ ]:
# Functions to save and load the library state to/from a file
def save_library_state(library):
    """
    This function saves the state of the Library object using Python's pickle module.
    Serialization like this allows the program to store objects persistently (saving them to a file),
    so that the data can be reloaded and used again later.
    """
    with open(LIBRARY_FILE, 'wb') as f:
        pickle.dump(library, f)

def load_library_state():
    """
    This function loads the state of the Library object from a file. If the file doesn't exist, it returns
    a new instance of the Library. This allows the program to continue where it left off after being restarted.
    
    This demonstrates file handling in Python and the ability to save and reload complex objects.
    """
    try:
        with open(LIBRARY_FILE, 'rb') as f:
            return pickle.load(f)
    except (FileNotFoundError, EOFError):
        return Library() # If the file does not exist or is empty, create a new Library instance


In [ ]:
# Global function to display the menu and interact with the Library system
def menu():
    """
    This function is the main interface for interacting with the library system. It displays a menu of
    options that the user can choose from, and then calls the appropriate functions based on the user's choice.
    
    The 'menu' function is also an example of code modularity, where various smaller functions
    (like 'add_book', 'display_books', 'borrow_book') are called based on the user's input.
    
    The library state is loaded from the file at the start and saved to the file before exiting.
    """
    library = load_library_state()  # Load the saved state of the library
    
    while True:
        # Displaying the menu options to the user
        print("\nWelcome to the Library Management System")
        print("1. Login")
        print("2. Add Book (Admin Only)")
        print("3. Display Books")
        print("4. Borrow Book (Members Only)")
        print("5. Return Book")
        print("6. Exit")
        
        choice = input("Please enter your choice: ")  # Get user's menu selection

        if choice == '1':  # Option to login a user
            username = input("Enter your username: ")
            library.login(username)

        elif choice == '2':  # Option to add a new book (Admin only)
            title = input("Enter the title of the book: ")
            author = input("Enter the author of the book: ")
            genre = input("Enter the genre of the book: ")
            library.add_book(title, author, genre)

        elif choice == '3':  # Option to display all available books
            library.display_books()

        elif choice == '4':  # Option to borrow a book (Members only)
            title = input("Enter the title of the book to borrow: ")
            library.borrow_book(title)

        elif choice == '5':  # Option to return a borrowed book
            title = input("Enter the title of the book to return: ")
            library.return_book(title)

        elif choice == '6':  # Option to exit the system
            save_library_state(library)  # Save the state before exiting
            print("Library state saved. Goodbye!")
            break

        else:
            print("Invalid choice! Please enter a number between 1 and 6.")





In [ ]:
# Load or create the library instance
library = load_library_state()

# Add a user (Admin or Member)
library.add_user("adminkunal", "Admin")  # Adding an Admin user
library.add_user("memberkunal", "Member")  # Adding a Member user

# Optionally, you can save the state after adding the users
save_library_state(library)


In [ ]:
# To start the library system, you can just call menu() in any Jupyter cell
menu()

In [ ]:
menu()